In [1]:
import pickle
import json
import mmcv
from tqdm import tqdm_notebook as tqdm
from mmdet.datasets import build_dataloader, build_dataset
from pycocotools.coco import COCO
import pycocotools.coco as cocoapi
import pycocotools.mask as MASK
import os
import cv2
from icecream import ic
import matplotlib.pyplot as plt
from mmdet.models import build_detector
from mmcv.runner import load_checkpoint, get_dist_info
import numpy as np
from multiprocessing import Pool

In [25]:

cfg = mmcv.Config.fromfile("configs/rs_cascade_mask_rcnn_r50_fpn_ohem.py")
CLASSES = ('tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane','ground-track-field',
               'helicopter', 'airport','harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field',
               'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad')
results = mmcv.load("./result/val.pkl")
dataset = build_dataset(cfg.data.test)
img_infos = dataset.load_annotations("/home/xfr/mmdetection/data/rscup/annotation/annos_rscup_val.json")
ann = {}
last_name = ""
last = ""
for i in tqdm(range(len(results))):
    filename = img_infos[i]['filename']
    items = filename.split("_")
    name = items[0]
    img = cv2.imread("./data/rscup/val/" + filename)
    h, w, _ = img.shape
    if name not in ann:
        ann[name] = {"bbox": [[] for i in range(len(dataset.CLASSES))], "seg": [[] for i in range(len(dataset.CLASSES))],
                     "vis": [[] for i in range(len(dataset.CLASSES))], "rect": [[] for i in range(len(dataset.CLASSES))]}
    scale_factor = 1/float(items[1])
    locx = int(float(items[2]))
    locy = int(float(items[3]))
    result = results[i]
    bboxes = result[0]
    segs = result[1]
    for cls in range(len(dataset.CLASSES)):
        bbox_cls = bboxes[cls]
        seg_cls = segs[cls]
        if( len(bbox_cls) > 0):
            for bbox, rle in zip(bbox_cls, seg_cls):
                xmin, ymin, xmax, ymax, score = bbox
#                 cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0,0,255), 2)
#                 cv2.putText(img, str(cls), (int(xmin), int(ymin)), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)

                xmin += locx
                ymin += locy
                xmax += locx
                ymax += locy
                bounding_box = np.array([xmin*scale_factor, ymin*scale_factor, xmax*scale_factor, ymax*scale_factor, score])
                mask = MASK.decode(rle)     
                contours, hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,
                                                                 cv2.CHAIN_APPROX_SIMPLE)
                areas = [cv2.contourArea(contour)  for contour in contours]
                if(len(areas) > 0):
                    index = np.argmax(areas)
                    contour = contours[index]
                    rect = cv2.minAreaRect(contour)
                    box = cv2.boxPoints(rect)
                    box += np.array([locx, locy])
                    
                    
                    center = np.array(rect[0])
                    size = np.array(rect[1])
                    angel = rect[2]
                    size *= scale_factor
                    center += np.array([locx, locy])
                    center *= scale_factor
                    rect = (tuple(center), tuple(size), angel )
                else:
                    continue
                ann[name]['rect'][cls].append(rect)
                ann[name]["bbox"][cls].append(bounding_box)
                ann[name]['vis'][cls].append(np.array([box[0]*scale_factor, box[1]*scale_factor, box[2]*scale_factor, box[3]*scale_factor]))
#                 poly = []
#                 for point in box:
#                     poly.append(point[0])
#                     poly.append(point[1])
#                 poly = np.array(poly, np.double)
#                 poly = [poly*scale_factor]
#                 rles = MASK.frPyObjects(poly, H, W)
#                 rle = MASK.merge(rles)
#                 ann[name]["seg"][cls].append(rle)
    #cv2.imwrite("./vis/"+filename, img)
print(len(ann.keys()))
    #coco_eval(result_file, eval_types, dataset.coco)
    # nms
    # toBbox
    # box - mask - rle
    # rles = maskUtils.frPyObjects(segm, h, w)
       #     rle = maskUtils.merge(rles)
mmcv.dump(ann, "./result/eval_temp.pkl")

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


593


In [18]:
def get_ann(bboxes, segs, cls, name, locx, locy, scale_factor):
    Rect = []
    Bbox = []
    Vis = []
    bbox_cls = bboxes[cls]
    seg_cls = segs[cls]
    if( len(bbox_cls) > 0):
        for bbox, rle in zip(bbox_cls, seg_cls):
            xmin, ymin, xmax, ymax, score = bbox
            xmin += locx
            ymin += locy
            xmax += locx
            ymax += locy
            bounding_box = np.array([xmin*scale_factor, ymin*scale_factor, xmax*scale_factor, ymax*scale_factor, score])
            mask = MASK.decode(rle)     
            contours, hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,
                                                             cv2.CHAIN_APPROX_SIMPLE)
            areas = [cv2.contourArea(contour)  for contour in contours]
            if(len(areas) > 0):
                index = np.argmax(areas)
                contour = contours[index]
                rect = cv2.minAreaRect(contour)
                box = cv2.boxPoints(rect)
                box += np.array([locx, locy])


                center = np.array(rect[0])
                size = np.array(rect[1])
                angel = rect[2]
                size *= scale_factor
                center += np.array([locx, locy])
                center *= scale_factor
                rect = (tuple(center), tuple(size), angel )
            else:
                continue
            Rect.append(rect)
            Bbox.append(bounding_box)
            Vis.append(np.array([box[0]*scale_factor, box[1]*scale_factor, box[2]*scale_factor, box[3]*scale_factor]))
    return  cls, name, Rect, Bbox, Vis
def get_all_ann(filename, result,size, CLASS_NUM=18):
    items = filename.split("_")
    name = items[0]
    h, w = size
#     img = cv2.imread("./data/rscup/val/" + filename)
#     h, w, _ = img.shape
    scale_factor = 1/float(items[1])
    locx = int(float(items[2]))
    locy = int(float(items[3]))
    bboxes = result[0]
    segs = result[1]
    Rect = [[] for cls in range(CLASS_NUM)]
    Bbox = [[] for cls in range(CLASS_NUM)]
    Vis = [[] for cls in range(CLASS_NUM)]
    for cls in range(CLASS_NUM):
        bbox_cls = bboxes[cls]
        seg_cls = segs[cls]
        if( len(bbox_cls) > 0):
            for bbox, rle in zip(bbox_cls, seg_cls):
                xmin, ymin, xmax, ymax, score = bbox
                xmin += locx
                ymin += locy
                xmax += locx
                ymax += locy
                bounding_box = np.array([xmin*scale_factor, ymin*scale_factor, xmax*scale_factor, ymax*scale_factor, score])
                mask = MASK.decode(rle)     
                contours, hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,
                                                                 cv2.CHAIN_APPROX_SIMPLE)
                areas = [cv2.contourArea(contour)  for contour in contours]
                if(len(areas) > 0):
                    index = np.argmax(areas)
                    contour = contours[index]
                    rect = cv2.minAreaRect(contour)
                    box = cv2.boxPoints(rect)
                    box += np.array([locx, locy])


                    center = np.array(rect[0])
                    size = np.array(rect[1])
                    angel = rect[2]
                    size *= scale_factor
                    center += np.array([locx, locy])
                    center *= scale_factor
                    rect = (tuple(center), tuple(size), angel )
                else:
                    continue
                Rect[cls].append(rect)
                Bbox[cls].append(bounding_box)
                Vis[cls].append(np.array([box[0]*scale_factor, box[1]*scale_factor, box[2]*scale_factor, box[3]*scale_factor]))
    return  name, Rect, Bbox, Vis
cfg = mmcv.Config.fromfile("configs/rs_cascade_mask_rcnn_r50_fpn_ohem.py")
CLASSES = ('tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane','ground-track-field',
               'helicopter', 'airport','harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field',
               'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad')
results = mmcv.load("./result/val.pkl")
dataset = build_dataset(cfg.data.test)
img_infos = dataset.load_annotations("/home/xfr/mmdetection/data/rscup/annotation/annos_rscup_val.json")
ann = {}
rets= []
CLASS_NUM=18
p = Pool(32)
def update(*a):
    pbar.update()
pbar = tqdm(total=40000)
for i in tqdm(range(len(results))):
    filename = img_infos[i]['filename']
    h = img_infos[i]['height']
    w = img_infos[i]['width']
    rets.append(p.apply_async(get_all_ann, args=(filename, results[i],(h, w), 18), callback=update)) 

for ret in rets:
    name, Rect, Bbox, Vis = ret.get()
    if name not in ann:
        ann[name] = {"bbox": [[] for i in range(CLASS_NUM)], "vis": [[] for i in range(CLASS_NUM)], 
                     "rect": [[] for i in range(CLASS_NUM)]}
    for cls in range(CLASS_NUM):
        ann[name]['bbox'][cls] += Bbox[cls]
        ann[name]['rect'][cls] += Rect[cls]
        ann[name]['vis'][cls] += Vis[cls]
mmcv.dump(ann, "./result/eval2_temp.pkl")
p.close()
p.join()

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


In [5]:
def nms_rotate_cpu(boxes, scores, iou_threshold, max_output_size):
    keep = []
    order = scores.argsort()[::-1]
    num = len(boxes)
    suppressed = np.zeros((num), dtype=np.int)
    for _i in range(num):
        if len(keep) >= max_output_size:
            break
        i = order[_i]
        if suppressed[i] == 1:
            continue
        keep.append(i)
        r1 = boxes[i]
        r1 = tuple(r1)
        area_r1 = boxes[i][1][0] * boxes[i][1][1]
        for _j in range(_i + 1, num):
            j = order[_j]
            if suppressed[i] == 1:
                continue
            r2 = boxes[j]
            area_r2 = boxes[j][1][0] * boxes[j][1][1]
            inter = 0.0
            r2 = tuple(r2)
            int_pts = cv2.rotatedRectangleIntersection(r1, r2)[1]
            if int_pts is not None:
                order_pts = cv2.convexHull(int_pts, returnPoints=True)
                int_area = cv2.contourArea(order_pts)
                inter = int_area * 1.0 / (area_r1 + area_r2 - int_area + 0.0000001)
              #  inter = int_area * 1.0 / min(area_r1+1e-5, area_r2+1e-5)

            if inter >= iou_threshold:
                suppressed[j] = 1

    return np.array(keep, np.int64)
def rotate_nms(ann, thresh):
    for name in tqdm(ann.keys()):
        info = ann[name]
        for cls in range(len(dataset.CLASSES)):
            bbox = np.array(info['bbox'][cls], np.float32)
            seg = np.array(info['seg'][cls])
            vis = np.array(info['vis'][cls])
            rect = info['rect'][cls]
            if(len(bbox)<=0):
                continue
            scores = bbox[:, 4]
            inds = nms_rotate_cpu(rect, scores, thresh, 1000)
            ann[name]['bbox'][cls] = bbox[inds]
            ann[name]['seg'][cls] = seg[inds]
            ann[name]['vis'][cls] = vis[inds]
#             ann[name]['rect'] =  rect[inds]
    return ann

In [20]:
def py_cpu_nms_poly_fast(dets, thresh):
    obbs = dets[:, 0:-1]
    x1 = np.min(obbs[:, 0::2], axis=1)
    y1 = np.min(obbs[:, 1::2], axis=1)
    x2 = np.max(obbs[:, 0::2], axis=1)
    y2 = np.max(obbs[:, 1::2], axis=1)
    scores = dets[:, 8]
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    polys = []
    for i in range(len(dets)):
        tm_polygon = polyiou.VectorDouble([dets[i][0], dets[i][1],
                                            dets[i][2], dets[i][3],
                                            dets[i][4], dets[i][5],
                                            dets[i][6], dets[i][7]])
        polys.append(tm_polygon)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        ovr = []
        i = order[0]
        keep.append(i)
        # if order.size == 0:
        #     break
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        # w = np.maximum(0.0, xx2 - xx1 + 1)
        # h = np.maximum(0.0, yy2 - yy1 + 1)
        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        hbb_inter = w * h
        hbb_ovr = hbb_inter / (areas[i] + areas[order[1:]] - hbb_inter)
        # h_keep_inds = np.where(hbb_ovr == 0)[0]
        h_inds = np.where(hbb_ovr > 0)[0]
        tmp_order = order[h_inds + 1]
        for j in range(tmp_order.size):
            iou = polyiou.iou_poly(polys[i], polys[tmp_order[j]])
            hbb_ovr[h_inds[j]] = iou
            # ovr.append(iou)
            # ovr_index.append(tmp_order[j])

        # ovr = np.array(ovr)
        # ovr_index = np.array(ovr_index)
        # print('ovr: ', ovr)
        # print('thresh: ', thresh)
        try:
            if math.isnan(ovr[0]):
                pdb.set_trace()
        except:
            pass
        inds = np.where(hbb_ovr <= thresh)[0]

        # order_obb = ovr_index[inds]
        # print('inds: ', inds)
        # order_hbb = order[h_keep_inds + 1]
        order = order[inds + 1]
        # pdb.set_trace()
        # order = np.concatenate((order_obb, order_hbb), axis=0).astype(np.int)
    return keep


#### poly nms

In [8]:
from mmdet.ops.nms import nms_wrapper
def nms(ann, thresh):
    for name in tqdm(ann.keys()):
        info = ann[name]
        for cls in range(18):
            bbox = np.array(info['bbox'][cls], np.float32)
            vis = np.array(info['vis'][cls])
            if(len(bbox)<=0):
                continue
            dets = vis.reshape(-1, 8)
            dets = np.array(dets, np.int32)
            scores = bbox[:, 4]
            dets = np.c_[dets, scores]
            #print(bbox.shape)
            inds = py_cpu_nms_poly_fast(dets, thresh)
           # print(len(inds))
            ann[name]['bbox'][cls] = bbox[inds]
            ann[name]['vis'][cls] = vis[inds]
    return ann
thresh=0.5
# result = copy.deepcopy(ann)
ann = mmcv.load("./result/eval2_temp.pkl")
result2 = nms(ann, thresh)
mmcv.dump(result2, "./result/eval2_postnms.pkl")

NameError: name 'py_cpu_nms_poly_fast' is not defined

#### bbx nms

In [19]:
from mmdet.ops.nms import nms_wrapper
def nms(ann, thresh):
    for name in tqdm(ann.keys()):
        info = ann[name]
        for cls in range(18):
            bbox = np.array(info['bbox'][cls], np.float32)
            vis = np.array(info['vis'][cls])
            if(len(bbox)<=0):
                continue
            #print(bbox.shape)
            _, inds = nms_wrapper.nms(bbox, thresh)
           # print(len(inds))
            ann[name]['bbox'][cls] = bbox[inds]
            ann[name]['vis'][cls] = vis[inds]
    return ann
thresh=0.5
# result = copy.deepcopy(ann)
ann = mmcv.load("./result/eval2_temp.pkl")
result2 = nms(ann, thresh)
mmcv.dump(result2, "./result/eval2_postnms.pkl")

### generate file

In [20]:
result2 = mmcv.load("./result/eval2_postnms.pkl")
names = list(result2.keys())
res = { CLASSES[cls]: [] for cls in range(len(dataset.CLASSES)) }
for name in tqdm(names):
    result = result2[name]
    bboxes = result["bbox"]
    segs = result['vis']
    for cls in range(len(dataset.CLASSES)):
        curr_class = CLASSES[cls]
        bbox_cls = bboxes[cls]
        seg_cls = segs[cls]
        if( len(bbox_cls) > 0):
            for bbox, rle in zip(bbox_cls, seg_cls):
                rle = np.array(rle)
                xmin, ymin, xmax, ymax, score = bbox
                location = list(rle.flatten())
                location = [str(int(x)) for x in location]
                if(score<0.05):
                    continue
                out = name + " "+str(score)+ " "+" ".join(location)
                res[curr_class].append(out)
for key in res.keys():
    print(key)
    fp = open("./result/val_temp/"+key+".txt", 'w')
    print(len(res[key]))
    for line in res[key]:
        fp.write(line+"\n")
    fp.close()

tennis-court
1151
container-crane
71
storage-tank
4250
baseball-diamond
638
plane
3961
ground-track-field
670
helicopter
293
airport
211
harbor
5835
ship
19647
large-vehicle
15893
swimming-pool
1892
soccer-ball-field
484
roundabout
865
basketball-court
515
bridge
2304
small-vehicle
58979
helipad
1


In [13]:
from shapely.geometry import *
import poly.polyiou as polyiou
def voc_ap(rec, prec, use_07_metric=False):
    """ ap = voc_ap(rec, prec, [use_07_metric])
      Compute VOC AP given precision and recall.
      If use_07_metric is true, uses the
      VOC 07 11 point method (default:False).
    """
    if use_07_metric:
        # 11 point metric
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
        ap = ap + p / 11.
    else:
        # correct AP calculation
        # first append sentinel values at the end
        # first appicend sentinel values at the end
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        # compute the precision envelope
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        # to calculate area under PR curve, look for points
        # where X axis (recall) changes value
        i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap
def no_overlap(a, b):
    xmin, xmax, ymin, ymax = a
    gxmin, gxmax, gymin, gymax = b
    if(xmax < gxmin or  ymin > gymax or gxmax < xmin or gymin > ymax):
        return True
    else:
        return False
def eval(classname, detpath, ovthresh, coco):
    CLASS=['tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad']
    #CLASS={'tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad'}
    class_to_ind = dict(zip(CLASS, range(len(CLASS))))
    imgIds = coco.getImgIds()
    recs = {}
    use_07_metric = False
    for imgid in imgIds:
        img = coco.loadImgs(imgid)[0]
        file_name = img['file_name']
        file_name = file_name.split(".")[0]
        annIds = coco.getAnnIds(imgIds=[imgid], iscrowd=None)
        anns = coco.loadAnns(annIds)
        objects = []
        for ann in anns:
            obj = {}
            obj['name'] = CLASS[ann['category_id']]
            obj['bbox'] = ann['segmentation']
            objects.append(obj)
        recs[file_name] = objects
    class_recs = {}
    npos = 0
    for filename in list(recs.keys()):
        R = [obj for obj in recs[filename] if obj['name'] == classname]
        bbox = np.array([x['bbox'] for x in R])
        bbox = np.reshape(np.squeeze(bbox),(-1,4, 2))
        det = [False] * len(R)
        npos = npos + len(bbox)
        class_recs[filename] = {'bbox': bbox,
                                 'det': det}
    detfile = detpath.format(classname)
    with open(detfile, 'r') as f:
        lines = f.readlines()

    splitlines = [x.strip().split(' ') for x in lines]
    image_ids = [x[0] for x in splitlines]
    confidence = np.array([float(x[1]) for x in splitlines])
    BB = np.array([[float(z) for z in x[2:]] for x in splitlines])
    BB = np.reshape(BB, (-1, 4, 2))
    nd = len(image_ids)
    tp = np.zeros(nd)
    fp = np.zeros(nd)
    if BB.shape[0] > 0:
        # sort by confidence
        sorted_ind = np.argsort(-confidence)
        sorted_scores = np.sort(-confidence)
        BB = BB[sorted_ind, :]
        image_ids = [image_ids[x] for x in sorted_ind]

        # go down dets and mark TPs and FPs
        for d in tqdm(range(nd)):
            R = class_recs[image_ids[d]]
            bb = BB[d, :].astype(float).flatten()
            ovmax = -np.inf
            BBGT = R['bbox'].astype(float).reshape(-1,8)
            if BBGT.size > 0:
                # compute overlaps
                # intersection
                BBGT_xmin =  np.min(BBGT[:, 0::2], axis=1)
                BBGT_ymin = np.min(BBGT[:, 1::2], axis=1)
                BBGT_xmax = np.max(BBGT[:, 0::2], axis=1)
                BBGT_ymax = np.max(BBGT[:, 1::2], axis=1)
                bb_xmin = np.min(bb[0::2])
                bb_ymin = np.min(bb[1::2])
                bb_xmax = np.max(bb[0::2])
                bb_ymax = np.max(bb[1::2])

                ixmin = np.maximum(BBGT_xmin, bb_xmin)
                iymin = np.maximum(BBGT_ymin, bb_ymin)
                ixmax = np.minimum(BBGT_xmax, bb_xmax)
                iymax = np.minimum(BBGT_ymax, bb_ymax)
                iw = np.maximum(ixmax - ixmin + 1., 0.)
                ih = np.maximum(iymax - iymin + 1., 0.)
                inters = iw * ih

                # union
                uni = ((bb_xmax - bb_xmin + 1.) * (bb_ymax - bb_ymin + 1.) +
                       (BBGT_xmax - BBGT_xmin + 1.) *
                       (BBGT_ymax - BBGT_ymin + 1.) - inters)

                overlaps = inters / uni

                BBGT_keep_mask = overlaps > 0
                BBGT_keep = BBGT[BBGT_keep_mask, :]
                BBGT_keep_index = np.where(overlaps > 0)[0]
                
                
                
                def calcoverlaps(BBGT_keep, bb):
                    overlaps = []
                    for index, GT in enumerate(BBGT_keep):

                        overlap = polyiou.iou_poly(polyiou.VectorDouble(BBGT_keep[index]), polyiou.VectorDouble(bb))
                        overlaps.append(overlap)
                    return overlaps
                if len(BBGT_keep) > 0:
                    overlaps = calcoverlaps(BBGT_keep, bb)

                    ovmax = np.max(overlaps)
                    jmax = np.argmax(overlaps)
                    # pdb.set_trace()
                    jmax = BBGT_keep_index[jmax]

            if ovmax > ovthresh:
                if not R['det'][jmax]:
                    tp[d] = 1.
                    R['det'][jmax] = 1
                else:
                    fp[d] = 1.
            else:
                fp[d] = 1.
    fp = np.cumsum(fp)
    tp = np.cumsum(tp)
    rec = tp / float(npos)
    # avoid divide by zero in case the first detection matches a difficult
    # ground truth
    prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
    ap = voc_ap(rec, prec, use_07_metric)
    print("ap of {} is {}".format(classname, ap))
    return ap

In [21]:
from multiprocessing import Pool
p.close()
p.join()
p = Pool(18)
detpath="./result/val_temp/{}.txt"
CLASS=['tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad']
ovthresh = 0.5
aps = []
coco=COCO("/home/xfr/rssid/data/annotation/annos_rscup_val.json")
for classname in tqdm(CLASS):
    aps.append(p.apply_async(eval, args=(classname, detpath, ovthresh, coco)))
#     ap = eval(classname, detpath, ovthresh, coco)
#     ret.append(ap)
ret = []
for ap in aps:
    ret.append(ap.get())
print("map is {}".format(np.mean(np.array(ret))))
p.close()
p.join()

loading annotations into memory...
Done (t=5.37s)
creating index...
index created!


ap of container-crane is 0.0
ap of tennis-court is 0.9471871184162259
ap of helicopter is 0.5587836717732454
ap of ground-track-field is 0.7358625689552859
ap of baseball-diamond is 0.7624642192899145
ap of airport is 0.7976206570793456
ap of swimming-pool is 0.5597168047738967
ap of storage-tank is 0.6314881477347973
ap of plane is 0.8977405909719963
ap of harbor is 0.7361267138683685
ap of large-vehicle is 0.7679770589594814
ap of roundabout is 0.6299256796308623
ap of bridge is 0.4567274136151177
ap of soccer-ball-field is 0.5836499359971897
ap of basketball-court is 0.6849244231323451
ap of ship is 0.8601033785809702
ap of helipad is 0.0
ap of small-vehicle is 0.4676732700743925
map is 0.6154428696029686


In [15]:
result2 = mmcv.load("./result/eval_postnms.pkl")
ann = result2
cfg = mmcv.Config.fromfile("configs/rs_test.py")
dataset = build_dataset(cfg.data.val)
for idx in tqdm(range(len(dataset))):
    img_info = dataset.img_infos[idx]
    img = cv2.imread("/home/xfr/rssid/data/val/images/"+img_info['filename'])
    filename = img_info['filename'].split(".")[0]
    info = ann[filename]
    seg = info['vis']
    box = info['bbox']
    for cls in range(len(dataset.CLASSES)):
        polys = seg[cls]
        boxes = box[cls]
        for poly, b in zip(polys, boxes):
            temp = np.array([poly], np.int32)
            cv2.putText(img, CLASSES[cls], (int(b[0]), int(b[1])), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.polylines(img, temp, 1, (0, 255, 0), 1)
    cv2.imwrite("./result/val/{}".format(filename)+".jpg", img)

loading annotations into memory...
Done (t=0.42s)
creating index...
index created!


In [77]:
import mmcv
import numpy as np
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval


def coco_eval(result_files, result_types, coco, max_dets=(100, 300, 1000)):
    for res_type in result_types:
        assert res_type in [
            'proposal', 'proposal_fast', 'bbox', 'segm', 'keypoints'
        ]

    if mmcv.is_str(coco):
        coco = COCO(coco)
    assert isinstance(coco, COCO)

    if result_types == ['proposal_fast']:
        ar = fast_eval_recall(result_files, coco, np.array(max_dets))
        for i, num in enumerate(max_dets):
            print('AR@{}\t= {:.4f}'.format(num, ar[i]))
        return

    for res_type in result_types:
        result_file = result_files[res_type]
        assert result_file.endswith('.json')

        coco_dets = coco.loadRes(result_file)
        
        img_ids = coco.getImgIds()
        iou_type = 'bbox' if res_type == 'proposal' else res_type
        cocoEval = COCOeval(coco, coco_dets, iou_type)
#         for cls in range(18):
#             cocoEval.params.catIds = [cls]
#             print(cocoEval.params)
#             print("------------------------------------------",CLASSES[cls],"----------------------------------------------")
        cocoEval.params.imgIds = img_ids
        cocoEval.params.maxDets = [100, 100, 100]
        if res_type == 'proposal':
            cocoEval.params.useCats = 0
            cocoEval.params.maxDets = list(max_dets)

        cocoEval.evaluate()
        cocoEval.accumulate()
        cocoEval.summarize()


In [47]:
ann = mmcv.load("./result/eval_temp.pkl")
cfg = mmcv.Config.fromfile("configs/rs_test.py")
dataset = build_dataset(cfg.data.val)
for idx in tqdm(range(len(dataset))):
    img_info = dataset.img_infos[idx]
    img = cv2.imread("/home/xfr/rssid/data/val/images/"+img_info['filename'])
    filename = img_info['filename'].split(".")[0]
    info = ann[filename]
    seg = info['vis']
    for cls in range(len(dataset.CLASSES)):
        polys = seg[cls]
        for poly in polys:
            temp = np.array([poly], np.int32)
            cv2.polylines(img, temp, 1, (0, 255, 0), 1)
    cv2.imwrite("./result/prenms/{}".format(filename)+".jpg", img)

loading annotations into memory...
Done (t=0.46s)
creating index...
index created!


In [78]:
from mmdet.core import results2json
outputs = []
ann = mmcv.load("postnms.pkl")
cfg = mmcv.Config.fromfile("configs/rs_test.py")
dataset = build_dataset(cfg.data.val)
for idx in range(len(dataset)):
    img_info = dataset.img_infos[idx]
    filename = img_info['filename'].split(".")[0]
    if filename not in ann:
        #print(filename)
        ann[filename] = {"bbox": [np.array([]) for i in range(len(dataset.CLASSES))], "seg":[[] for i in range(len(dataset.CLASSES))]}
    info = ann[filename]
    #print(info['seg'][0])
    m = np.array(info['bbox'])
    for i in range(len(m)):
        m[i] = np.array(m[i])
    temp = (m, info['seg'])
    outputs.append(temp)
result_files = results2json(dataset, outputs, "xx")
coco_eval(result_files, ['segm'], dataset.coco)

loading annotations into memory...
Done (t=1.44s)
creating index...
index created!
Loading and preparing results...
DONE (t=2.70s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=184.62s).
Accumulating evaluation results...
DONE (t=1.54s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.327
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.564
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.338
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.196
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.357
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.478
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.398
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.398
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDe

In [17]:
detpath="./result/val_temp/{}.txt"
CLASS=['tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad']
ovthresh = 0.5
aps = []
coco=COCO("/home/xfr/rssid/data/annotation/annos_rscup_val.json")
for classname in tqdm(CLASS):
    ap = eval(classname, detpath, ovthresh, coco)
    print("ap of {} is {}".format(classname, ap))
    aps.append(ap)
aps = np.array(aps)
print("map is {}".format(np.mean(aps)))

loading annotations into memory...
Done (t=1.20s)
creating index...
index created!


ap of tennis-court is 0.9461078239129896
ap of tennis-court is 0.9461078239129896


ap of container-crane is 0.011904761904761904
ap of container-crane is 0.011904761904761904


ap of storage-tank is 0.6423659993901099
ap of storage-tank is 0.6423659993901099


ap of baseball-diamond is 0.8416521148635843
ap of baseball-diamond is 0.8416521148635843


...

ap of soccer-ball-field is 0.6242170684009745
ap of soccer-ball-field is 0.6242170684009745


ap of roundabout is 0.6920844814776893
ap of roundabout is 0.6920844814776893


ap of basketball-court is 0.6834833805051985
ap of basketball-court is 0.6834833805051985


ap of bridge is 0.4487948537678508
ap of bridge is 0.4487948537678508


ap of small-vehicle is 0.4905447846177097
ap of small-vehicle is 0.4905447846177097
ap of helipad is 0.0
ap of helipad is 0.0

map is 0.6240618615440766


In [18]:
from multiprocessing import Pool
p = Pool()
for i in range(5):
    p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')

NameError: name 'long_time_task' is not defined

In [ ]:
from shapely.geometry import *
import copy
import time
def py_cpu_pnms(dets, pts, thresh):
    dets = np.array(dets)
    pts = np.array(pts)
    scores = dets[:, 4]
    order = scores.argsort()[::-1]
    areas = np.zeros(scores.shape)
    inter_areas = np.zeros((scores.shape[0], scores.shape[0]))
    for il in range(len(pts)):
        poly = Polygon(pts[il])
        areas[il] = poly.area
        for jl in range(il, len(pts)):
            polyj = Polygon(pts[jl])
            inS = poly.intersection(polyj)
            inter_areas[il][jl] = inS.area
            inter_areas[jl][il] = inS.area

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        ovr = inter_areas[i][order[1:]] / (areas[i] + areas[order[1:]] - inter_areas[i][order[1:]])
        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]
    return np.array(keep, np.int32)
def nms(ann, thresh):
    
    for name in tqdm(ann.keys()):
        info = ann[name]
        for cls in range(len(dataset.CLASSES)):
            bbox = np.array(info['bbox'][cls])
           
            seg = np.array(info['seg'][cls])
            vis = np.array(info['vis'][cls])
            if(len(bbox)<=0):
                continue
            t1 = time.time()
            h, w = seg[0]['size']
            bbox_result = []
            seg_result = []
            vis_result = []
            tm = time.time()
            maps = np.zeros((h, w))
            for box in bbox:
                xmin, ymin, xmax, ymax , score = box
                maps[int(ymin):int(ymax), int(xmin):int(xmax)] = 1
#             for a in seg:
#                 m = MASK.decode(a)
#                 maps.append(m)
#             maps = np.sum(np.array(maps), axis=0)
#             maps[maps>=1] = 1
            tn = time.time()
            kernel = np.ones((5,5),np.uint8) 
            maps = cv2.dilate(maps,kernel,iterations = 1)
            contours, hierarchy = cv2.findContours((maps).astype(np.uint8), cv2.RETR_TREE,
                                                                 cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                xmin, ymin, w, h = cv2.boundingRect(contour)
                xmax = xmin+w
                ymax = ymin+h
                region_box = []
                region_mask = []
                region_vis = []
                for i, bb in enumerate(bbox):
                    if(bb[0]>=xmin and bb[1]>=ymin and bb[2]<=xmax and bb[3]<=ymax):
                        region_box.append(bbox[i])
                        region_mask.append(seg[i])
                        region_vis.append(vis[i])
                if(len(region_box) <= 0):
                    continue
                keep_ind = py_cpu_pnms(region_box, region_vis, thresh)
                bbox_result += list(np.array(region_box)[keep_ind])
                seg_result += list(np.array(region_mask)[keep_ind])
                vis_result += list(np.array(region_vis)[keep_ind])
            ann[name]['bbox'][cls] = bbox_result
            ann[name]['seg'][cls] = seg_result
            ann[name]['vis'][cls] = vis_result
            t2 = time.time()
                
    return ann
thresh=0.5
# result = copy.deepcopy(ann)
ann = mmcv.load("eval_temp.pkl")
result = nms(ann, thresh)


In [17]:
cfg = mmcv.Config.fromfile("configs/rs_cascade_mask_rcnn_r50_fpn_ohem.py")
CLASSES = ('tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane','ground-track-field',
               'helicopter', 'airport','harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field',
               'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad')
results = mmcv.load("./result/val.pkl")
dataset = build_dataset(cfg.data.test)
img_infos = dataset.load_annotations("/home/xfr/mmdetection/data/rscup/annotation/annos_rscup_val.json")
ann = {}
last_name = ""
last = ""
img_size = {}
for i in tqdm(range(len(results))):
    filename = img_infos[i]['filename']
    print(img_infos[i])
    items = filename.split("_")
    name = items[0]
    img = cv2.imread("./data/rscup/val/" + filename)
    h, w, _ = img.shape
    

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


{'license': 1, 'file_name': 'P0814_0.5_226.0_0.0_0part1.jpg', 'coco_url': 'xxx', 'height': 275, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 0, 'filename': 'P0814_0.5_226.0_0.0_0part1.jpg'}
{'license': 1, 'file_name': 'P0814_0.5_0.0_0.0_0part2.jpg', 'coco_url': 'xxx', 'height': 275, 'width': 512, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 1, 'filename': 'P0814_0.5_0.0_0.0_0part2.jpg'}
{'license': 1, 'file_name': 'P0814_0.5_226.0_0.0_0part3.jpg', 'coco_url': 'xxx', 'height': 275, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 2, 'filename': 'P0814_0.5_226.0_0.0_0part3.jpg'}
{'license': 1, 'file_name': 'P0814_0.5_0.0_0.0_0part4.jpg', 'coco_url': 'xxx', 'height': 275, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 3, 'filename': 'P0814_0.5_0.0_0.0_0part4.jpg'}
{'license': 1, 'file_name': 'P0814_1_964.0_0.0_0part1.jpg', 'coco_url': 'xxx', 'height': 512, 'width': 511, 'date_captured': '2019-06-25', '

{'license': 1, 'file_name': 'P1314_1_1248.0_1664.0_0part35.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 75, 'filename': 'P1314_1_1248.0_1664.0_0part35.jpg'}
{'license': 1, 'file_name': 'P1314_1_1248.0_2080.0_0part36.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 76, 'filename': 'P1314_1_1248.0_2080.0_0part36.jpg'}
{'license': 1, 'file_name': 'P1314_1_1248.0_2496.0_0part37.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 77, 'filename': 'P1314_1_1248.0_2496.0_0part37.jpg'}
{'license': 1, 'file_name': 'P1314_1_1248.0_2912.0_0part38.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 78, 'filename': 'P1314_1_1248.0_2912.0_0part38.jpg'}
{'license': 1, 'file_name': 'P1314_1_1248.0_3328.0_0part39.jpg', 'coco_url': 'xxx', 'height': 511, 'widt

{'license': 1, 'file_name': 'P1275_0.5_832.0_832.0_0part14.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 156, 'filename': 'P1275_0.5_832.0_832.0_0part14.jpg'}
{'license': 1, 'file_name': 'P1275_0.5_832.0_1248.0_0part15.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 157, 'filename': 'P1275_0.5_832.0_1248.0_0part15.jpg'}
{'license': 1, 'file_name': 'P1275_0.5_1248.0_0.0_0part16.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 158, 'filename': 'P1275_0.5_1248.0_0.0_0part16.jpg'}
{'license': 1, 'file_name': 'P1275_0.5_1248.0_416.0_0part17.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 159, 'filename': 'P1275_0.5_1248.0_416.0_0part17.jpg'}
{'license': 1, 'file_name': 'P1275_0.5_1248.0_832.0_0part18.jpg', 'coco_url': 'xxx', 'height': 511

{'license': 1, 'file_name': 'P1275_1_1664.0_2912.0_0part47.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 216, 'filename': 'P1275_1_1664.0_2912.0_0part47.jpg'}
{'license': 1, 'file_name': 'P1275_1_1664.0_3328.0_0part48.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 217, 'filename': 'P1275_1_1664.0_3328.0_0part48.jpg'}
{'license': 1, 'file_name': 'P1275_1_2080.0_0.0_0part49.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 218, 'filename': 'P1275_1_2080.0_0.0_0part49.jpg'}
{'license': 1, 'file_name': 'P1275_1_2080.0_416.0_0part50.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 219, 'filename': 'P1275_1_2080.0_416.0_0part50.jpg'}
{'license': 1, 'file_name': 'P1275_1_2080.0_832.0_0part51.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 5

...

{'license': 1, 'file_name': 'P4151_0.5_1664.0_0.0_0part24.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 516, 'filename': 'P4151_0.5_1664.0_0.0_0part24.jpg'}
{'license': 1, 'file_name': 'P4151_0.5_1664.0_416.0_0part25.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 517, 'filename': 'P4151_0.5_1664.0_416.0_0part25.jpg'}
{'license': 1, 'file_name': 'P4151_0.5_1664.0_832.0_0part26.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 518, 'filename': 'P4151_0.5_1664.0_832.0_0part26.jpg'}
{'license': 1, 'file_name': 'P4151_0.5_1664.0_1248.0_0part27.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 519, 'filename': 'P4151_0.5_1664.0_1248.0_0part27.jpg'}
{'license': 1, 'file_name': 'P4151_0.5_1664.0_1664.0_0part28.jpg', 'coco_url': 'xxx', 'height'

{'license': 1, 'file_name': 'P4151_1_416.0_0.0_0part15.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 563, 'filename': 'P4151_1_416.0_0.0_0part15.jpg'}
{'license': 1, 'file_name': 'P4151_1_416.0_416.0_0part16.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 564, 'filename': 'P4151_1_416.0_416.0_0part16.jpg'}
{'license': 1, 'file_name': 'P4151_1_416.0_832.0_0part17.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 565, 'filename': 'P4151_1_416.0_832.0_0part17.jpg'}
{'license': 1, 'file_name': 'P4151_1_416.0_1248.0_0part18.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 566, 'filename': 'P4151_1_416.0_1248.0_0part18.jpg'}
{'license': 1, 'file_name': 'P4151_1_416.0_1664.0_0part19.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_

{'license': 1, 'file_name': 'P4151_1_2080.0_2080.0_0part64.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 612, 'filename': 'P4151_1_2080.0_2080.0_0part64.jpg'}
{'license': 1, 'file_name': 'P4151_1_2080.0_2496.0_0part65.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 613, 'filename': 'P4151_1_2080.0_2496.0_0part65.jpg'}
{'license': 1, 'file_name': 'P4151_1_2080.0_2912.0_0part66.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 614, 'filename': 'P4151_1_2080.0_2912.0_0part66.jpg'}
{'license': 1, 'file_name': 'P4151_1_2080.0_3328.0_0part67.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 615, 'filename': 'P4151_1_2080.0_3328.0_0part67.jpg'}
{'license': 1, 'file_name': 'P4151_1_2080.0_3744.0_0part68.jpg', 'coco_url': 'xxx', 'height': 511, '

{'license': 1, 'file_name': 'P4151_1_4160.0_832.0_0part116.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 664, 'filename': 'P4151_1_4160.0_832.0_0part116.jpg'}
{'license': 1, 'file_name': 'P4151_1_4160.0_1248.0_0part117.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 665, 'filename': 'P4151_1_4160.0_1248.0_0part117.jpg'}
{'license': 1, 'file_name': 'P4151_1_4160.0_1664.0_0part118.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 666, 'filename': 'P4151_1_4160.0_1664.0_0part118.jpg'}
{'license': 1, 'file_name': 'P4151_1_4160.0_2080.0_0part119.jpg', 'coco_url': 'xxx', 'height': 511, 'width': 511, 'date_captured': '2019-06-25', 'flickr_url': 'xxx', 'id': 667, 'filename': 'P4151_1_4160.0_2080.0_0part119.jpg'}
{'license': 1, 'file_name': 'P4151_1_4160.0_2496.0_0part120.jpg', 'coco_url': 'xxx', 'height':

KeyboardInterrupt: 